In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
os.environ['PROJ_LIB'] = '/opt/conda/share/proj/'
n_cpus = os.cpu_count()
n_threads = n_cpus
os.environ['OMPI_MCA_rmaps_base_oversubscribe'] = '1'
os.environ['USE_PYGEOS'] = '0'
os.environ['PROJ_LIB'] = '/opt/conda/share/proj/'
os.environ['NUMEXPR_MAX_THREADS'] = f'{n_threads}'
os.environ['NUMEXPR_NUM_THREADS'] = f'{n_threads}'
os.environ['OMP_THREAD_LIMIT'] = f'{n_threads}'
os.environ["OPENBLAS_NUM_THREADS"] = f'{n_threads}' 
os.environ["MKL_NUM_THREADS"] = f'{n_threads}'
os.environ["VECLIB_MAXIMUM_THREADS"] = f'{n_threads}'
os.environ["OMP_DYNAMIC"] = f'TRUE'
os.environ['TREELITE_BIND_THREADS'] = '0'
os.environ["OMP_NUM_THREADS"] = f'{n_threads}'
os.environ["OPENBLAS_NUM_THREADS"] = f'{n_threads}'
os.environ["MKL_NUM_THREADS"] = f'{n_threads}'
os.environ["NUMEXPR_NUM_THREADS"] = f'{n_threads}'
import pathlib
import tl2cgen
import treelite
import joblib
import numpy as np
import skmap_bindings as sb
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
import treelite.sklearn
import warnings
import numpy as np
import time
import shutil
import subprocess

folder_path = '/mnt/ripley/global_soc/scikit-map/global-soil-mapping'
# prop_list = ['ocd', 'soc', 'bulk.density', 'ph.h2o' ,'coarse', 'texture1', 'texture2']
prop_list = ['texture1', 'texture2']

transforms_dict = {
    'ocd': 'log1p',
    'soc': 'log1p',
    'bulk.density': None,
    'ph.h2o': None,
    'coarse': 'log1p',
    'texture1': None,
    'texture2': None
}
# version = "20250204"
version = "20250523"
tmp_folder = 'tmp_compile'
os.makedirs(tmp_folder, exist_ok=True)

out_folder = f'/mnt/slurm/jobs/global_soc'
nthread = 96

# generate the production model with all the data we have

In [2]:
for prop in prop_list:
    print(f'\n{prop}--------------------------------------------------------------')
    
    
    in_path = f'{folder_path}/production/model_rf.{prop}_production_v{version}.joblib'
    covs_path = f'{out_folder}/model_rf.{prop}_production_v{version}.covs'
    output_path = f'{out_folder}/model_rf.{prop}_production_v{version}.so'
    output_path_joblib = f'{out_folder}/model_rf.{prop}_production_v{version}.joblib'
    os.makedirs(tmp_folder, exist_ok=True)


            
    model_rf = joblib.load(in_path)
    treelite_model = treelite.sklearn.import_model(model_rf)
    with open(covs_path, "w") as file:
        for item in list(model_rf.feature_names_in_):
            file.write(item + "\n")

    tl2cgen.generate_c_code(treelite_model, tmp_folder, {"verbose": 1,"quantize": 1,"parallel_comp": 96}, verbose=True)
    temp_libpath = tl2cgen.create_shared('/opt/intel/oneapi/compiler/2024.0/bin/icx-cc -O3 -xCORE-AVX512 -qopt-zmm-usage=high',
                                         tmp_folder,
                                         nthread = nthread,
                                         verbose = True,
                                         long_build_time_warning = False)
    try:
        subprocess.run(["cp", temp_libpath, output_path], check=True)
        print("File 1 copied successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error copying file: {e}")
    try:
        subprocess.run(["cp", in_path, output_path_joblib], check=True)
        print("File 2 copied successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error copying file: {e}")
    shutil.rmtree(tmp_folder)



texture1--------------------------------------------------------------
[09:22:35] ../src/compiler/ast_native.cc:663: Using ASTNativeCompiler
[09:22:38] ../src/compiler/ast/builder/split.cc:30: Parallel compilation enabled; member trees will be divided into 96 translation units.
[09:23:05] ../src/c_api/c_api.cc:105: Code generation finished. Writing code to files...
[09:23:05] ../src/c_api/c_api.cc:110: Writing file recipe.json...
[09:23:05] ../src/c_api/c_api.cc:110: Writing file tu62.c...
[09:23:05] ../src/c_api/c_api.cc:110: Writing file tu61.c...
[09:23:05] ../src/c_api/c_api.cc:110: Writing file tu60.c...
[09:23:05] ../src/c_api/c_api.cc:110: Writing file tu57.c...
[09:23:05] ../src/c_api/c_api.cc:110: Writing file tu23.c...
[09:23:05] ../src/c_api/c_api.cc:110: Writing file tu47.c...
[09:23:05] ../src/c_api/c_api.cc:110: Writing file tu26.c...
[09:23:05] ../src/c_api/c_api.cc:110: Writing file tu19.c...
[09:23:05] ../src/c_api/c_api.cc:110: Writing file tu21.c...
[09:23:05] ../sr